In [1]:
import cv2
import os   
import numpy as np
import pywt
import matplotlib.pyplot as plt
from skimage.filters import gabor
from scipy.signal import convolve2d

In [2]:
# Lista de nombres de archivo de imágenes en el directorio
nombres_imagenes_normales = os.listdir('./data')

In [61]:
def aplicar_fft_a_imagen(imagen, umbral_suavizado=0.1):
    # Aplicar la transformada de Fourier 2D
    imagen_fft = np.fft.fft2(imagen)

    # Desplazar el componente de baja frecuencia al centro
    imagen_fft_desplazada = np.fft.fftshift(imagen_fft)

    # Calcular la magnitud del espectro de frecuencias
    magnitud_fft = np.abs(imagen_fft_desplazada)

    # Normalizar la magnitud para que esté en el rango [0, 1]
    magnitud_fft_normalized = magnitud_fft / np.max(magnitud_fft)

    # Aplicar el umbral de suavizado en la magnitud normalizada
    magnitud_fft_suavizada = magnitud_fft_normalized.copy()
    magnitud_fft_suavizada[magnitud_fft_normalized < umbral_suavizado] = 0

    # Crear un filtro de ventana para suavizar las transiciones
    filtro_ventana = cv2.getGaussianKernel(len(magnitud_fft_suavizada), 30)
    filtro_ventana = filtro_ventana * filtro_ventana.T
    magnitud_fft_suavizada = magnitud_fft_suavizada * filtro_ventana

    # Obtener la imagen suavizada en el dominio de la frecuencia
    imagen_fft_suavizada = imagen_fft_desplazada * np.exp(1j * np.angle(imagen_fft_desplazada)) * magnitud_fft_suavizada

    # Deshacer el desplazamiento
    imagen_fft_suavizada = np.fft.ifftshift(imagen_fft_suavizada)

    # Aplicar la transformada inversa de Fourier
    imagen_suavizada = np.fft.ifft2(imagen_fft_suavizada).real

    return magnitud_fft_suavizada, imagen_suavizada

def aplicar_wav_a_imagen(imagen, wavelet='haar', kernel=2, level=1, sigma=0.1):
    # Convertir la imagen a escala de grises si es una imagen a color
    if len(imagen.shape) == 3:
        imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)

    # Aplicar 2D Discrete Wavelet Transform
    coeffs = pywt.wavedec2(imagen, wavelet, level=level)

    # Obtener las dimensiones de la imagen wavelet
    rows, cols = imagen.shape

    # Realizar la convolución directamente en coeffs[0]
    coeffs[0] = convol(coeffs[0], kernel)

    # Reconstruir la imagen filtrada
    imagen_filtrada = pywt.waverec2(coeffs, wavelet)

    return imagen_filtrada

def filtro_paso_alto(imagen, kernel_size=3):
    # Convierte la imagen a escala de grises si no lo está
    if len(imagen.shape) == 3:
        imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)

    # Aplica un filtro de suavizado (puede ser un filtro gaussiano)
    suavizado = cv2.GaussianBlur(imagen, (kernel_size, kernel_size), 0)

    # Aplica el filtro de paso alto (resta la imagen suavizada de la original)
    paso_alto = imagen - suavizado

    return paso_alto

def filtro_paso_bajo(imagen, kernel_size=3):
    # Aplica un filtro gaussiano
    suavizado = cv2.GaussianBlur(imagen, (kernel_size, kernel_size), 0)

    return suavizado

def convol(imagen, k = 5):
    # Define el kernel (por ejemplo, un filtro promedio de 3x3)
    kernel = np.ones((k, k), dtype=float) / 9

    # Convierte la imagen a escala de grises si no lo está
    if len(imagen.shape) == 3:
        imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)

    # Normaliza la imagen al rango [0, 1]
    imagen = imagen.astype(float) / 255.0

    # Asegúrate de que la imagen sea 2-D
    if len(imagen.shape) > 2:
        imagen = np.mean(imagen, axis=-1)

    # Aplica la convolución usando scipy.signal.convolve2d
    resultado = convolve2d(imagen, kernel, mode='same', boundary='symm')

    # Escala el resultado al rango [0, 255]
    resultado = (resultado * 255).astype(np.uint8)

    return resultado

def filtrar_ruido_gabor(imagen, frecuencia, orientacion, tamano_kernel):
    # Crea un filtro de Gabor
    filtro_gabor = cv2.getGaborKernel((tamano_kernel, tamano_kernel), frecuencia, orientacion, 10.0, 0.5, 0, ktype=cv2.CV_32F)

    # Aplica el filtro de Gabor a la imagen
    imagen_filtrada = cv2.filter2D(imagen, cv2.CV_8UC3, filtro_gabor)

    return imagen_filtrada


In [62]:
# Procesar cada imagen en el directorio
for nombre_imagen in nombres_imagenes_normales:
    ruta_imagen = os.path.join("./data", nombre_imagen)

    # Cargar la imagen utilizando OpenCV
    imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)  # Convierte la imagen a escala de grises

    # Aplicar la transformada de Fourier a la imagen
    imagen_fft = aplicar_fft_a_imagen(imagen)

    # Guardar la imagen procesada en un nuevo archivo
    nombre_imagen_salida = f'fft_{nombre_imagen}'
    ruta_imagen_salida = os.path.join("./Preprocesado_fft/", nombre_imagen_salida)
    cv2.imwrite(ruta_imagen_salida, imagen_fft[1])

print("Procesamiento de imágenes completado.")

Procesamiento de imágenes completado.


In [51]:
# Wavelets
for nombre_imagen in nombres_imagenes_normales:
    ruta_imagen = os.path.join("./data", nombre_imagen)

    # Cargar la imagen utilizando OpenCV
    imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)  # Convierte la imagen a escala de grises

    # Aplicar la transformada de Fourier a la imagen
    imagen_wav = aplicar_wav_a_imagen(imagen)

    # Guardar la imagen procesada en un nuevo archivo
    nombre_imagen_salida = f'wav_{nombre_imagen}'
    ruta_imagen_salida = os.path.join("./Preprocesado_wav/", nombre_imagen_salida)
    cv2.imwrite(ruta_imagen_salida, imagen_wav)

print("Procesamiento de imágenes completado.")

Procesamiento de imágenes completado.


In [58]:
# Gabor
for nombre_imagen in nombres_imagenes_normales:
    ruta_imagen = os.path.join("./data", nombre_imagen)

    # Cargar la imagen utilizando OpenCV
    imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)  # Convierte la imagen a escala de grises

    # Aplicar la transformada de Fourier a la imagen
    imagen_gabor = filtrar_ruido_gabor(imagen,0.5,0,3)

    # Guardar la imagen procesada en un nuevo archivo
    nombre_imagen_salida = f'gabor_{nombre_imagen}'
    ruta_imagen_salida = os.path.join("./Preprocesado_gabor/", nombre_imagen_salida)
    cv2.imwrite(ruta_imagen_salida, imagen_gabor)

print("Procesamiento de imágenes completado.")

Procesamiento de imágenes completado.


In [6]:
#Con paso alto
for nombre_imagen in nombres_imagenes_normales:
    ruta_imagen = os.path.join("./data", nombre_imagen)

    # Cargar la imagen utilizando OpenCV
    imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)  # Convierte la imagen a escala de grises
    imagen_nueva = filtro_paso_alto(imagen)
    imagen_nueva2 = filtro_paso_bajo(imagen) 

    # Guardar la imagen procesada en un nuevo archivo
    nombre_imagen_salida = f'alto_{nombre_imagen}'
    ruta_imagen_salida = os.path.join("./Preprocesado_alto/", nombre_imagen_salida)
    cv2.imwrite(ruta_imagen_salida, np.abs(imagen_nueva))

    nombre_imagen_salida = f'bajo_{nombre_imagen}'
    ruta_imagen_salida = os.path.join("./Preprocesado_bajo/", nombre_imagen_salida)
    cv2.imwrite(ruta_imagen_salida, np.abs(imagen_nueva2))

In [8]:
#Convolucionar
for nombre_imagen in nombres_imagenes_normales:
    ruta_imagen = os.path.join("./data", nombre_imagen)

    # Cargar la imagen utilizando OpenCV
    imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)  # Convierte la imagen a escala de grises
    imagen_nueva = convol(imagen)

    # Guardar la imagen procesada en un nuevo archivo
    nombre_imagen_salida = f'con_{nombre_imagen}'
    ruta_imagen_salida = os.path.join("./Preprocesado_con/", nombre_imagen_salida)
    cv2.imwrite(ruta_imagen_salida, np.abs(imagen_nueva))